<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Affine layer

Affine층은 fully-connected layer, dense layer이라고도 불리며 입력된 벡터에 가중치 행렬(weight matrix)를 곱하고 편향(bias)를 더하는 연산을 수행한다. 딥러닝 모델이 역전파(back propagation)을 통해 학습을 진행하면 각 가중치와 편향은 손실함수(loss function)을 최소화하는 최적값을 찾아가게 된다.
\
\
\
Forward

$Y = XW +B$

$ \begin{pmatrix} y_{1} & y_{2} & y_{3} \\ \vdots & \vdots & \vdots \end{pmatrix} = \begin{pmatrix} x_{1} & x_{2} \\ \vdots & \vdots \end{pmatrix} \begin{pmatrix} w_{11} & w_{12} & w_{13} \\ w_{21} & w_{22} & w_{23} \end{pmatrix}  + \begin{pmatrix} b_{1} & b_{2} & b_{3} \\ \vdots & \vdots & \vdots \end{pmatrix} $

여기서 입력값 X는 2차원 벡터이고 출력값 Y은 3차원벡터이며 위의 수식은 입력값 X의 배치처리를 고려해 확장한 수식이다. 확장된 X행렬은 각행이 배치된 입력값이며 확장된 B행렬은 편향벡터를 배치 수 만큼 각 행에 반복한 행렬이다.(Broadcasting)
\
\
\
Backward

Affine층의 계산그래프는 X와 W의 행렬곱, 그 결과값과 B의 덧셈으로 이루어진다. 주의할 점은 배치처리시 B의 덧셈연산 이전에 Repeat연산(편향벡터를 배치 수 만큼 각행에 반복하는 것)이 추가로 진행된다는 점 이다.
\
\
\
1.덧셈노드의 역전파

흘러들어온 미분이 $\frac{\partial L}{\partial Y}$ 일때 $Y = XW + B$임을 생각해 L에 대한 XW와 B에대한 편미분을 각각 구하면,

$\frac{\partial L}{\partial WX} = \frac{\partial L}{\partial Y} \frac{\partial Y}{\partial WX}$ 이므로 $\frac{\partial L}{\partial WX} = \frac{\partial L}{\partial Y} * 1$ 이 되고,

$\frac{\partial L}{\partial B} = \frac{\partial L}{\partial Y} \frac{\partial Y}{\partial B}$ 이므로 $\frac{\partial L}{\partial B} = \frac{\partial L}{\partial Y} * 1$ 이 된다.
\
\
\
2.행렬곱노드의 역전파

W방향으로의 역전파를 진행한다. 먼저 순전파를 생각하면,

$ \begin{pmatrix} x_1 & x_2 \end{pmatrix} \begin{pmatrix} w_{11} & w_{12} & w_{13} \\ w_{21} & w_{22} & w_{23} \end{pmatrix} = \begin{pmatrix} x_1 w_{11} + x_2 w_{21} & x_1 w_{12} + x_2 w_{22} & x_1 w_{13} + x_2 w_{23} \end{pmatrix} $ 이다. 그리고 이 과정을 8개의 변수를 갖는 다변수함수 $F()$라고 생각하면 $F()$ 에 대한 각 변수의 편미분을 구할 수 있다. $ w_{11} $을 예로들면,

$ \frac{\partial F()}{\partial w_{11}} = \begin{pmatrix} x_1 \\
0 \\ 0 \end{pmatrix} $ 이다.

여기서 흘러들어온 미분이 $ \frac{\partial L}{\partial F()} = \begin{pmatrix}
d_1 & d_2 & d_3 \end{pmatrix} $ 일때 연쇄법칙을 사용하면 손실함수에 대한 $w_{11}$의 편미분을 구할 수 있다.

$ \frac{\partial L}{\partial w_{11}} = \frac{\partial L}{\partial F()} \frac{\partial F()}{\partial w_{11}} = \begin{pmatrix}
d_1 & d_2 & d_3 \end{pmatrix} \begin{pmatrix} x_1 \\
0 \\ 0 \end{pmatrix} = d_1 x_1 $

위 과정을 되풀이하여 손실함수에대한 각 가중치의 편미분을 구하면,

$ \begin{pmatrix} d_1 x_1 & d_2 x_1 & d_3 x_1 \\ d_1 x_2 & d_2 x_2 & d_3 x_2 \end{pmatrix}$ 이 되고 이는 $  \begin{pmatrix} x_1 \\ x_2 \end{pmatrix} \begin{pmatrix}d_1 & d_2 & d_3 \end{pmatrix} $ 와 같으므로 최종적으로
\
\
$ \frac{\partial L}{\partial W} = X^{T} \frac{\partial L}{\partial F()} $ 이 된다.

위와 같은 방식으로 X방향으로의 역전파를 진행하면,

$ \frac{\partial L}{\partial X} = \frac{\partial L}{\partial F()} W^{T} $ 이 된다.
\
\
\
3.repeat노드의 역전파

!! repeat노드의 인풋을 벡터로하고 chain rule로 재현하려하면 shape이 맞지 않아 진행이 안됨.

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W # 가중치 행렬
        self.b = b # bias 벡터

        # 입력된 data 초기화
        self.x = None
        self.original_x_shape = None
        # 가중치와 bias 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1) # 4차원 텐서를 행렬로 변환. -1을 입력하면 reshape 가능한 숫자를 알아서 대입한다.
        self.x = x

        out = np.dot(self.x, self.W) + self.b # affine 변환. bias의 경우 배치처리를 할경우 행렬뎃섬에서 shape이 맞지 않는데 numpy에서 자동으로 벡터를 복사해 각 열에 더하게 된다.

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T) # data방향 역전파.
        self.dW = np.dot(self.x.T, dout) # weight matrix 방향 역전파.
        self.db = np.sum(dout, axis=0) # bias 방향 역전파. 행렬의 경우 axis 0: 행, 1: 열. 텐서의 경우 axis 0: 각 행렬, 1: 각 행렬의 행, 2: 각 행렬의 열

        dx =dx.reshape(*self.original_x_shape) # 입력데이터. *은 ()와 ,를 없에주지만 없어도 되는 것 같음.

        return dx # 데이터방향으로 미분값을 흘려보내야 다음 노드에서 값을 받아 역전파를 수행할 수 있음.